In [ ]:
#1º Passo – Importar as bibliotecas que serão utilizadas

from gradcam import GradCAM
from PIL import Image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils
import numpy as np
import argparse
import imutils
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
#2º Passo – Carregar os arrays construídos na etapa referente ao pré-processamento de dados do Modelo 2 e dividir os dados em dados de treinamento e dados de teste

X_train = np.load("/content/drive/My Drive/Python/COVID/Arrays/Modelo2/X_Train.npy")

Y_train = np.load("/content/drive/My Drive/Python/COVID/Arrays/Modelo2/Y_Train.npy")

X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train, test_size = 0.2, random_state = 40)

In [ ]:
#3º Passo – Construir e salvar o mapa de ativação de classe (CAM)
for image in range(X_test.shape[0]):
  img = Image.fromarray(X_test[image])
  img.save(f"imagens_{image}.png")


for array in range(X_test.shape[0]):
  orig = cv2.imread(f"imagens_{array}.png")
  orig = cv2.resize(orig, (237, 237))
  image = load_img((f"imagens_{array}.png"), target_size=(237, 237))
  image = img_to_array(image)
  image = image/255
  image = np.expand_dims(image, axis=0)
  preds = model.predict(image)
  i = np.argmax(preds[0])
  if i == 0:
    label = f"COVID-19: {int(preds[0][i] * 100)}%"
  elif i == 1:
    label = f"NORMAL: {int(preds[0][i] * 100)}%"
  else:
    label = f"O.INF. : {int(preds[0][i] * 100)}%"
  print("[INFO] {}".format(label))
  cam = GradCAM(model, i)


  heatmap = cam.compute_heatmap(image)
  heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
  (heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)
  cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
  cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,0.8, (255, 255, 255), 2)
  # #output = np.vstack([orig, heatmap, output])
  output = np.vstack([output])

  print(orig.shape,heatmap.shape,output.shape)

  # #output = imutils.resize(output, height=700)
  print(output.shape)
  cv2_imshow(output)
  cv2.imwrite(f"/content/drive/My Drive/Python/COVID/Arrays/Modelo2/image_{array}.png",output)
  # #cv2.waitKey(0)